### Práctica 1. Resolución de problemas con búsqueda 
## Inteligencia Artificial I    2023/2024

Parte 1: AIMA 
Este notebook está organizado en 3 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.

Debes entender el comportamiento de la búsqueda en los distintos problemas. 
Observa la representación y el comportamiento de los distintos algoritmos. 
No hay que entregar este notebook pero se realizará un test para evaluar la comprensión de los conceptos. 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en representar estados y acciones, la comprobación de objetivo y el coste de las acciones.
La siguiente clase Problem (de la librería AIMA) representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problem, y requerirá implementar
* estado_inicial, es_estado_final(_), acciones(_), aplica(_,_), coste_de_aplicar_accion y eventualmente __init__ 



!git clone https://github.com/aimacode/aima-python.git

 _Para esta práctica se usarán los archivos _search.py y utils.py_ de AIMA_

In [ ]:
cd aima-python

In [ ]:
from search import *

In [ ]:
# El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 
from search import Problem

## Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver algunos ejemplos de cómo __definir un problema como subclase
de Problem__. En concreto, el problema de las jarras y el de los misioneros, vistos en clase.

In [ ]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs=list() 
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self,estado,accion):
    # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos para entender cómo funciona la clase Problem.

In [ ]:
# Estado inicial
p =Jarras()
p.initial

In [ ]:
# Acciones válidas desde el estado inicial 
p.actions(p.initial)

In [ ]:
# Resultado de aplicar una acción a un estado
p.result(p.initial,"llenar jarra de 4")

In [ ]:
# Comprobamos si un estado dado es objetivo
p.goal_test(p.initial)

### Problema de los misioneros

In [ ]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Aunque no es necesario puedes hacer tu propia implementación del problema 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [ ]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

In [ ]:
misioneros.initial

In [ ]:
# Acciones aplicables en el estado inicial
misioneros.actions(misioneros.initial)

Por ejemplo, para resolver el problema de los misioneros con 
el método de busqueda en anchura la llamada sería:  


In [ ]:
breadth_first_tree_search(misioneros).solution()

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

In [ ]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        pos_hueco = estado.index(0)  # busco la posicion del 0
        accs = list()
        if pos_hueco not in (0, 1, 2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0, 3, 6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (6, 7, 8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (2, 5, 8):
            accs.append("Mover hueco derecha")

        return accs

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        def swap(x):
            l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]
        if accion == "Mover hueco arriba":
            swap(-3)
        elif accion == "Mover hueco abajo":
            swap(3)
        elif accion == "Mover hueco izquierda":
            swap(-1)
        elif accion == "Mover hueco derecha":
            swap(1)

        return tuple(l)

    def h(self, node):
        """ Return the heuristic value for a given state."""
        return 1
    
    def coste_de_aplicar_accion(self, s, a):
        return 1

####  Prueba los siguientes ejemplos.

In [ ]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

In [ ]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [ ]:
p8.result(p8.initial,"Mover hueco arriba")

In [ ]:
p8.result(p8.initial,"Mover hueco abajo")

¿Por qué da error?
¿Deberíamos añadir controles adicionales para que no ocurra este error? 
¿Por qué no?

In [ ]:
p8.result(p8.initial,"Mover hueco derecha")

In [ ]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

### Usando la representación anterior representa tú el problema del puzle de 16 piezas.

In [ ]:
class Quince_Puzzle(Problem): 
    ## Completa la definición del problema
    ???? 

Realiza algunas pruebas para comprobar que la clase está bien definida

### Representa alguno de los problemas de la hoja de ejercicios. Por ejemplo piensa cómo representarías el siguiente:

Un grupo de 5 personas quiere cruzar un viejo y estrecho puente. Es una noche cerrada y se necesita llevar una linterna para cruzar. El grupo solo dispone de una linterna, a la que le quedan 5 minutos de batería.
1.	Cada persona tarda en cruzar 10, 30, 60, 80 y 120 segundos, respectivamente.
2.	El puente solo resiste un máximo de 2 personas cruzando a la vez, y cuando cruzan dos personas juntas, caminan a la velocidad del más lento.
3.	No se puede lanzar la linterna de un extremo a otro del puente, así que cada vez que crucen dos personas, alguien tiene que volver a cruzar hacia atrás con la linterna a buscar a los compañeros que falten, y así hasta que hayan cruzado todos.


Se debe proponer una representación adecuada que permita a un agente resolverlo con distintos parámetros de entrada (tamaños, valores ,..), explicando las decisiones de representación

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [ ]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [ ]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [ ]:
depth_first_graph_search(Jarras()).solution()

#### Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Observa cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [ ]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

# breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

In [ ]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

In [ ]:
p8.goal

In [ ]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
breadth_first_tree_search(estado).solution()

In [ ]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
moves = breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']
s = (2, 4, 3, 1, 5, 6, 7, 8, 0)
show((2, 4, 3, 1, 5, 6, 7, 8, 0))
for action in moves:
    s = estado.result(s,action)
    print()
    show(s)

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]


Si el método de busqueda en anchura es completo.. ¿la ejecución anterior no debería terminar? ¿qué crees que está pasando?

Añade a la definición del problema Ocho_Puzzle la función solvability que comprueba si un estado inicial tiene solución.

En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución.

In [ ]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [ ]:
sol

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

In [ ]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [ ]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

In [ ]:
depth_first_graph_search(estado).solution()

In [ ]:
breadth_first_graph_search(estado).solution()

Observa los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega. 
¿Qué algoritmo es más eficiente? ¿encuentran la solución óptima? ¿son completos? ¿alguno no ha podido terminar la ejecución? ¿Por qué? 

#### Definición de heurísticas

In [ ]:
#### Para el problema de los misioneros utilizaremos alguna de las heurísticas vistas en clase. 


In [ ]:
#### Para el problema del puzle de 8 y de 16 vamos a definir las siguientes funciones heurísticas:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [ ]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self, estado):
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        pos_hueco = estado.index(0)  # busco la posicion del 0
        accs = list()
        if pos_hueco not in (0, 1, 2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0, 3, 6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (6, 7, 8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (2, 5, 8):
            accs.append("Mover hueco derecha")

        return accs

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        def swap(x):
            l[pos_hueco], l[pos_hueco+x] = l[pos_hueco+x], l[pos_hueco]
        if accion == "Mover hueco arriba":
            swap(-3)
        elif accion == "Mover hueco abajo":
            swap(3)
        elif accion == "Mover hueco izquierda":
            swap(-1)
        elif accion == "Mover hueco derecha":
            swap(1)

        return tuple(l)

    def h(self, node):
        """ Return the heuristic value for a given state."""
        return 1
    
    # Hemos añadido las heurísticas a la clase para poder acceder al atributo goal
    def linear(self, node):
        return sum(x != sol for x, sol in zip(node.state, self.goal))

    def manhattan(self, node):
        state = node.state
        mhd = sum(abs(x-y) for x,y in zip(state, self.goal))
        return mhd

    def sqrt_manhattan(self, node):
        import math
        mhd = self.manhattan(node)
        return math.sqrt(mhd)

    def max_heuristic(self, node):
        score1 = self.manhattan(node)
        score2 = self.linear(node)
        return max(score1, score2)

    def coste_de_aplicar_accion(self, s, a):
        return 1
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0  

#### Vamos a usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [ ]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

In [ ]:
astar_search(puzle,puzle.manhattan).solution()

In [ ]:
def show(s):
    print(" ".join(map(str,s[:3])))
    print(" ".join(map(str,s[3:6])))
    print(" ".join(map(str,s[6:])))

In [ ]:
show(puzle.initial)

In [ ]:
puzle.goal

In [ ]:
astar_search(puzle,puzle.linear).solution()

In [ ]:
astar_search(puzle,puzle.max_heuristic).solution()

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [ ]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [ ]:
%%timeit
astar_search(puzzle_1, puzzle_1.linear)
astar_search(puzzle_2, puzzle_2.linear)
astar_search(puzzle_3, puzzle_3.linear)

In [ ]:
%%timeit
astar_search(puzzle_1, puzzle_1.manhattan)
astar_search(puzzle_2, puzzle_2.manhattan)
astar_search(puzzle_3, puzzle_3.manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, puzzle_1.sqrt_manhattan)
astar_search(puzzle_2, puzzle_2.sqrt_manhattan)
astar_search(puzzle_3, puzzle_3.sqrt_manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, puzzle_1.max_heuristic)
astar_search(puzzle_2, puzzle_2.max_heuristic)
astar_search(puzzle_3, puzzle_3.max_heuristic)

####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.

## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



In [ ]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [ ]:
# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 
import math

def linear(node):
    goal = (1,2,3,4,5,6,7,8,0)
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])

def manhattan(node):
    l = list(node.state)
    mhd = 0
    for m in range(0,3):
        for n in range(0,3):
            num = l[m*3+n]
            i = num/3; j = num%3 - 1
            if num == 0:
                i = 2; j = 2;
            mhd += abs(i-m) + abs(j-n);
 
    return mhd
                
def sqrt_manhattan(node):
    state = node.state
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)        

In [ ]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [ ]:
p8p.initial

In [ ]:
p8p.goal

In [ ]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, puzzle_1.manhattan).solution()

In [ ]:
astar_search(p8, p8.manhattan).solution()

In [ ]:
astar_search(p8p, manhattan).solution()

In [ ]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [ ]:
e1 = (7,2,4,3,6,1,8,0,5)

In [ ]:
%%time
resuelve_ocho_puzzle(e1,astar_search,manhattan)

In [ ]:
resuelve_ocho_puzzle(e1,astar_search,manhattan)

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

### Resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

           E1              E2              E3             
           
     +---+---+---+   +---+---+---+   +---+---+---+    
     | 4 | 5 | 6 |   | 1 | 2 | 6 |   | H | 2 | 4 |    
     +---+---+---+   +---+---+---+   +---+---+---+    
     | 1 | 0 | 3 |   | 4 | 5 | 3 |   | 6 | 5 | 8 |    
     +---+---+---+   +---+---+---+   +---+---+---+    
     | 7 | 8 | 2 |   | 8 | 7 | 0 |   | 7 | 3 | 1 |    
     +---+---+---+   +---+---+---+   +---+---+---+    
     
Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
Hacerlo con la función resuelve_ocho_puzzle, para obtener, además de la solución, la longitud (el coste) de la solución  obtenida y el número de nodos analizados. 
Para cada búsqueda observa los resultados obtenidos y anotalos para poder compararlos (L, longitud de la solución, NA, nodos analizados, T, tiempo). Compara también las versiones graph search y tree search de los algoritmos.
 
    - Búsqueda en anchura 
    - Búsqueda en profundidad 
    - Búsqueda de coste uniforme 
    - Búsqueda Primero el mejor (con diferentes heurísticas) 
    - Búsqueda A* (con diferentes heurísticas) 
-------------------------------------------
Debes observar si los resultados experimentales coinciden con las distintas propiedades teóricas estudiadas en clase. Por ejemplo, ¿se observa que la heurística manhattan es realmente más informada que la heurística linear? ¿A* es un algoritmo óptimo? ¿cómo afecta el control de repetidos a la eficiencia? ¿y a la optimalidad? ¿y a la completitud? 

In [ ]:
e1 = (4,5,6,1,0,3,7,8,2)

In [ ]:
%%time
resuelve_ocho_puzzle(e1,astar_search,manhattan)

In [ ]:
%%time
resuelve_ocho_puzzle(e1,breadth_first_graph_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e1,breadth_first_tree_search)

In [ ]:
%%time
resuelve_ocho_puzzle(e1,astar_search,lineal)

In [ ]:
e2 = (1,2,6,4,5,3,8,7,0)

In [ ]:
%%time
resuelve_ocho_puzzle(e2,astar_search,manhattan)

In [ ]:
e3 = (0,2,4,6,5,8,7,3,1)

In [ ]:
%%time
resuelve_ocho_puzzle(e4,astar_search,manhattan)

#### Puedes resolver tambien usando distintas búsquedas y/o heurísticas, el problema del 16 puzzle para algunos  estados iniciales observando los resultados y la eficiencia de las distintas opciones. 

#### Resuelve utilizando búsqueda los problemas de clase que hayas representado.